In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

from statsmodels.nonparametric.smoothers_lowess import lowess

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sb
import pymannkendall as mk
from scipy.fft import fft, fftfreq
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.signal import correlate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### new global

In [2]:
#new_global = pd.read_csv('Complete_TAVG_summary.txt', skiprows=20, delimiter='\t')
new_global = pd.read_excel('new_global.xlsx')

In [3]:
new_global.head(3)

,Year,Annual Anomaly,Annual Unc.,Five-year Anomaly,Five-year Unc.
0,1750,-1.420,0.883,NaN,NaN
1,1751,-1.515,1.161,NaN,NaN
2,1753,-1.175,0.915,NaN,NaN


In [4]:
df = pd.read_excel('linear_02 0501.xlsx')

In [5]:
df

,level_0,index,irtemp mean,dmy float,temp all c,anom modern anom,irtemp meang
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
18297,603.0,2021-12-31 00:00:00,9.775000,2021.541667,NaN,NaN,8.535000
18298,604.0,NaN,NaN,NaN,-20.01,0.93,NaN
18299,605.0,2022-12-31 00:00:00,10.183333,2022.541667,NaN,NaN,8.943333
18300,606.0,2023-12-31 00:00:00,10.845455,2023.500000,NaN,NaN,9.605455


In [6]:
ox = pd.read_excel('ox.xlsx', skiprows=4, skipfooter=8)
ox

# skip 2020: no data

,Unnamed: 0,Jan,Feb,Mar,Apr,May,June,July,Aug,Sept,Oct,Nov,Dec,Annual,Unnamed: 14,YEAR,Winter,Spring,Summer,Autumn
0,1811,#,#,#,#,#,#,#,#,#,#,#,#,NaN,NaN,1811,NaN,NaN,NaN,NaN
1,1812,#,#,#,#,#,#,#,#,#,#,#,#,NaN,NaN,1812,NaN,NaN,NaN,NaN
2,1813,#,#,#,#,#,#,#,#,#,#,#,2.888889,NaN,NaN,1813,NaN,NaN,NaN,NaN
3,1814,-2.578938,1.714198,3.186579,10.561728,10.330944,13.019441,17.08184,15.848865,13.203704,8.828554,5.358025,5.494624,8.504130,NaN,1814,0.674716,8.026417,15.316715,9.130094
4,1815,-1.482677,4.502714,5.106355,7.513867,13.772581,15.361667,15.832258,16.382258,13.731367,8.144839,0.845833,0.941226,8.408932,NaN,1815,2.838220,8.797601,15.858728,7.574013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2015,5.020968,4.25,7.325806,10.186667,12.245161,15.636667,17.530645,17.135484,13.613333,11.493548,10.236667,10.8,11.335068,NaN,2015,4.935484,9.919211,16.767599,11.781183
205,2016,6.195161,5.898276,6.18871,8.485,13.51129,15.99,18.498387,18.525806,16.968333,11.191935,6.363333,6.270968,11.194262,NaN,2016,7.631146,9.395000,17.671398,11.507867
206,2017,4.009677,6.691071,9.717742,9.903333,14.309677,17.76,18.609677,16.995161,14.58,13.058065,7.075,5.414516,11.540959,NaN,2017,5.657239,11.310251,17.788280,11.571022
207,2018,6.030484,3.341071,5.814516,10.846667,14.314516,17.208333,21.048387,18.379032,15.245,11.470968,8.61,7.41129,11.696973,NaN,2018,4.928690,10.325233,18.878584,11.775323


In [7]:
ox = ox.iloc[:, :-7].copy()
cols = ox.columns.to_list()
cols[0] = 'Year'
ox.columns = cols

In [8]:
ox = ox.iloc[2:].copy()

In [9]:
ox.set_index('Year', drop=True, inplace=True)

In [10]:
ox.head(3)

,Jan,Feb,Mar,Apr,May,June,July,Aug,Sept,Oct,Nov,Dec
Year,,,,,,,,,,,,
1813,#,#,#,#,#,#,#,#,#,#,#,2.888889
1814,-2.578938,1.714198,3.186579,10.561728,10.330944,13.019441,17.08184,15.848865,13.203704,8.828554,5.358025,5.494624
1815,-1.482677,4.502714,5.106355,7.513867,13.772581,15.361667,15.832258,16.382258,13.731367,8.144839,0.845833,0.941226


In [11]:
ox_stack = ox.stack().to_frame(name='ox')
ox_dates = pd.to_datetime(
ox_stack.index.get_level_values(level=1).astype(str) + '-' + ox_stack.index.get_level_values(level=0).astype(str), format='mixed')
ox_stack.index = ox_dates

In [12]:
ox_stack['ox'].replace('#', np.nan, inplace=True)

In [13]:
px.line(ox_stack.resample('1Y').mean())

In [14]:
# mean of 1850-1900
mean_1850_1900 = ox_stack.loc[pd.to_datetime('1850-01-01'):pd.to_datetime('1900-12-12')].mean()
mean_1850_1900

ox    9.602943
dtype: float64

In [15]:
ox_stack_1Y = ox_stack.resample('1Y').mean()

In [16]:
ox_stack_annual_change = ox_stack_1Y - ox_stack_1Y.shift(1)

In [17]:
# G(t) means it's a function of time, so a continuous data series

global_temp = pd.read_csv('1850-2023.csv', skiprows=4)
global_temp

,Year,Anomaly
0,185001,-0.37
1,185002,-0.13
2,185003,-0.24
3,185004,-0.20
4,185005,-0.19
...,...,...
2082,202307,1.13
2083,202308,1.23
2084,202309,1.43
2085,202310,1.36


In [18]:
global_temp['Year'] = global_temp['Year'].astype(str)
global_temp['Year'] = pd.to_datetime(
                        global_temp['Year'].str.slice(start=4) + '-' + 
                        global_temp['Year'].str.slice(stop=4))
global_temp.set_index('Year', drop=True, inplace=True)

In [19]:
px.scatter(global_temp)

In [20]:
# only keep the dates from global where we have the oxford data: left join
ox_stack_global = ox_stack.merge(global_temp, left_index=True, right_index=True, how='left').dropna(how='all')

In [21]:
ox_stack_global_1Y = ox_stack_global.resample('1Y').mean()

In [22]:
ox_stack_global_1Y['ox_change'] = ox_stack_global_1Y['ox'] - ox_stack_global_1Y['ox'].shift(1)

In [23]:
new_global.index = pd.to_datetime(new_global['Year'].astype(str) + '-12-31')
ox_stack_global_1Y = ox_stack_global_1Y.merge(new_global, left_index=True, right_index=True, how='left')
ox_stack_global_1Y['Anomaly'] = ox_stack_global_1Y['Annual Anomaly']

In [24]:
ox_stack_global_1Y.drop(columns=[
    'Annual Anomaly','Annual Unc.',	'Five-year Anomaly', 'Five-year Unc.', 'Year'],
    inplace=True)

In [25]:
ox_stack_global_1Y.tail(3)

,ox,Anomaly,ox_change
2017-12-31,11.510327,1.304,0.336393
2018-12-31,11.643355,1.142,0.133029
2019-12-31,12.331190,1.340,0.687835


In [26]:
px.line(ox_stack_global_1Y)# why drop Annual Anomnaly from new_global? did you read the whole code of course not

# Anomaly is 1.14 - ok-ish from the raw data
# the smoothed value is not really smoothed, so play around with the values. you need to make the line follow the
# the raw data better
# then your smoothed 2018 data should roughly be around 1.2 

In [27]:
# may need to redo/ignore ox_change
# 1. 41-year lowess filter on ox
# 2. same for Anomaly (global)
# 3. local = m * smoothed global + c
# correlation ox and GMST: 0.45
# correlation between ox and smoothed GMST: 0.67
# correlation between both smoothed: 0.98

In [28]:
fit_ox_anomaly = px.scatter(ox_stack_global_1Y, x='Anomaly', y='ox', trendline='ols')
fit_ox_anomaly.show()

In [29]:
#px.get_trendline_results(fit_ox_anomaly).values[0][0].summary() # great p-value

In [30]:
#ox_dt = ox_lowess[:, 0].astype('datetime64[ns]')
#ox_dt = pd.to_datetime(ox_dt)

In [31]:
it, frac = 41, 0.25

# change number_of_nans to datetime merge later

ox_lowess = lowess(endog=ox_stack_global_1Y['ox'], 
            exog=ox_stack_global_1Y.index,
            it=it,
            frac=frac
            )

global_lowess = lowess(endog=ox_stack_global_1Y['Anomaly'],
                exog=ox_stack_global_1Y.index,
                it=it,
                frac=frac
                )

ox_lowess_df = pd.DataFrame(ox_lowess)
ox_lowess_df[0] = pd.to_datetime(ox_lowess_df[0].astype('datetime64[ns]') )
ox_lowess_df.columns = ['date', 'ox smoothed']
#ox_lowess_df.head(3)

global_lowess_df = pd.DataFrame(global_lowess)
global_lowess_df[0] = pd.to_datetime(global_lowess_df[0].astype('datetime64[ns]') )
global_lowess_df.columns = ['date', 'Anomaly smoothed']
#global_lowess_df.head(3)


In [32]:
ox_stack_global_1Y = ox_stack_global_1Y.merge(ox_lowess_df, 
                                            left_index=True, right_on='date', how='left' )

ox_stack_global_1Y = ox_stack_global_1Y.merge(global_lowess_df, 
                                            left_on='date', right_on='date', how='left')
                                            
ox_stack_global_1Y.set_index('date', inplace=True)

In [33]:
ox_stack_global_1Y.head(3)

,ox,Anomaly,ox_change,ox smoothed,Anomaly smoothed
date,,,,,
1813-12-31,2.888889,-0.948,NaN,9.226528,-0.923275
1814-12-31,8.504130,-1.041,5.615241,9.234122,-0.919776
1815-12-31,8.387691,-1.474,-0.116440,9.241334,-0.916308


In [34]:
#pd.Series(global_lowess[:, 0]) # 170 rows, matches Anomaly after dropna. without dropna = 207

In [35]:
# s: 1.19k, n=0.1k, snr=11.3
# # s: 1.73k, n=0.54k, snr=3.2

# visualize data, not fitting

px.scatter(
        #x=ox_lowess[:, 0], 
        #x=ox_stack_global_1Y.index,
        #y=[ox_lowess[:, 1], ox_stack_global_1Y['ox']],
        #y=[ox_stack_global_1Y['ox'],  ox_stack_global_1Y['Anomaly'] ]
        #trendline='ols'

        ox_stack_global_1Y,
        y=['ox', 'Anomaly', 'ox smoothed', 'Anomaly smoothed']
    )

In [36]:
# fitting

linear_fit = px.scatter(ox_stack_global_1Y,
                    x='Anomaly smoothed', y='ox smoothed', 
                    #x='Anomaly', y='ox',
                    trendline='ols')

params = px.get_trendline_results(linear_fit).iloc[0].values[0]
alpha, intercept = params.params[1], params.params[0]

ox_stack_global_1Y['ox calculated'] = alpha * ox_stack_global_1Y['Anomaly smoothed'] + intercept

print(np.array([alpha, intercept, params.rsquared]).round(2))

linear_fit.show()

[ 1.01 10.13  0.97]


In [37]:
px.line(ox_stack_global_1Y.loc[pd.to_datetime('1850-01-01'):],
            x=ox_stack_global_1Y.loc[pd.to_datetime('1850-01-01'):].index, 
            y=['ox', 'ox calculated']
            )

In [38]:
#px.get_trendline_results(linear_fit).iloc[0].values[1].summary()

In [39]:
#fit = px.get_trendline_results(linear_fit).iloc[0].values[1]
#alpha, intercept = fit.params[1], fit.params[0]
#alpha, intercept

In [40]:
# note:
# 2. change the lowess it and frac to match the paper well
# 1. organize the code so it runs from the top to the bottom properly
# 3. calculate the R2 for smoothed global against oxford and you should have 0.98
# 4. calculate the alpha and intercept. the code is above but just make sure they are reasonably accurate to the paper


In [41]:
#px.line(ox_stack_global_1Y, y='Anomaly')

## SNR

In [42]:
alpha, intercept # alpha in the paper is 1.45 +/- .25 (95 % CI), not too bad

# G2018 from the paper: 1.19 K


(1.0065044821539026, 10.12519313560093)

In [43]:
ox_stack_global_1Y 

,ox,Anomaly,ox_change,ox smoothed,Anomaly smoothed,ox calculated
date,,,,,,
1813-12-31,2.888889,-0.948,NaN,9.226528,-0.923275,9.195913
1814-12-31,8.504130,-1.041,5.615241,9.234122,-0.919776,9.199434
1815-12-31,8.387691,-1.474,-0.116440,9.241334,-0.916308,9.202925
1816-12-31,7.785517,-1.816,-0.602173,9.248198,-0.912848,9.206407
1817-12-31,9.071523,-1.553,1.286006,9.254722,-0.909401,9.209877
...,...,...,...,...,...,...
2015-12-31,11.289579,1.237,-0.472611,11.467999,1.167550,11.300337
2016-12-31,11.173933,1.431,-0.115645,11.503416,1.196501,11.329477
2017-12-31,11.510327,1.304,0.336393,11.538557,1.225372,11.358536


In [44]:
px.line(ox_stack_global_1Y,
y=['Anomaly', 'Anomaly smoothed'])


In [45]:
# linear['signal'] = alpha * linear['temp all c']
# linear['fitted'] = alpha * linear['temp all c'] + intercept
# noise = (linear['fitted'] - linear['signal']).std()
# linear['SNR'] = linear['signal'] / noise


linear['signal'] = alpha * linear['']
linear['fitted'] = alpha * linear[''] + intercept
noise = (linear['fitted'] - linear['signal']).std()
linear['SNR'] = linear['signal'] / noise
linear['SNR']

NameError: name 'linear' is not defined

In [ ]:
# relative to the mean of 1850–1900
# ox: α = 1.45 ± 0.25
# “signal” for Oxford is αG2018 = 1.72 ± 0.30 K
# “noise,” = 0.54K
# S/N ratio of 3.2 ± 0.5 # from hawkins paper

# fit an OLS with px.scatter
# get the slope and intercept: L(t) = alpha * G(t) + β
# signal is alpha * G
# noise: component is defined as the standard deviation of the residuals (L − ∝G)
# SNR: np.where

linear['SNR label'] = np.where(linear['SNR'] < 1, 'normal', np.nan)
linear['SNR label'] = np.where((linear['SNR']>=1) & (linear['SNR'] < 2), 'unusual', linear['SNR label'])
linear['SNR label'] = np.where((linear['SNR']>=2) & (linear['SNR'] < 3), 'unfamiliar', linear['SNR label'])
linear['SNR label'] = np.where(linear['SNR'] >= 3, 'unknown')

linear['SNR label'] = np.where((normal, 'normal'), (unusual, 'unusual'), (unfamiliar, 'unfamiliar'), (unknown, 'unknown'))


In [ ]:
# smoothing
ox_stack_global_1Y['ox smoothed'] = ox_lowess[:, 1]


number_of_nans = ox_stack_global_1Y['Anomaly'].isna().value_counts().loc[True]

# always assume all the nans are the top of the table

ox_stack_global_1Y['Anomaly smoothed'] = np.concatenate([
                                            [np.nan] * number_of_nans,
                                            global_lowess[:, 1]
                                        ])